In [34]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn import linear_model

## Working with industry data

In [51]:
industry_target = pd.read_csv('industry_target from sba.csv')
industry_data = pd.read_csv('industry_all from census.csv')
industry_index = pd.read_csv('Codes.csv')

### Let's start with mapping targets from SBA

In [56]:
# reading in the index file
industry_index = industry_index.rename(index=str, columns={"2017 NAICS US   Code": "NAICS"})
industry_index['NAICS'] = pd.to_numeric(industry_index['NAICS'])

In [57]:
industry_target=industry_target.merge(industry_index, on='NAICS', how='left')

In [58]:
# filtering for those firms of size less than 500 employees 
ind_state_target = industry_target.loc[industry_target['firm type'].isin(['firm', 'firms'])][['Year', 
                                                        'State', 'NAICS', '<500', '2017 NAICS US Title']]
ind_state_target['<500'] = pd.to_numeric(ind_state_target['<500'])

In [61]:
# create one for the US
ind_us_target = ind_state_target.groupby(['NAICS', 'Year']).sum()

In [39]:
ind_state_target.head()

,Year,State,NAICS,<500,2017 NAICS US Title
0,2011.0,Alabama,113,586,Forestry and Logging
1,2011.0,Alabama,114,22,"Fishing, Hunting and Trapping"
2,2011.0,Alabama,115,163,Support Activities for Agriculture and Forestry
3,2011.0,Alabama,211,18,Oil and Gas Extraction
4,2011.0,Alabama,212,78,Mining (except Oil and Gas)


In [62]:
ind_us_target.head()

<500
NAICS Year           
113   1998.0  13982.0
      1999.0  13444.0
      2000.0  13099.0
      2001.0  12561.0
      2002.0  12274.0

### Now we can take a look at the data from CENSUS

The feature description table:

|  Name | Type   |  Description |
|---|---|---|
|  STATE | 	C   | Geographic Area Code, FIPS State 2-digit codes  |
| NAICS  |  C   |  Industry Code; 6-digit, North American Industry Classification System (NAICS) |
|  ENTRSIZE |  C  |  Enterprise Employment Size Code |
|  INIT_ESTB |  N  |  Number of Initial Year Establishments |
|  INIT_EMPL |  N  | Initial Year Employment with Noise  |
|  INIT_EMPLFL_N |  C  | Initial Year Employment Noise Flag   |
| INIT_EMPFL_R  | C   |   Initial Year Employment Range Flag / Data Suppression Flag |
|  NETCHG_ESTB |  N |  Change in Establishments |
| NETCHG_EMPL   |  N |  Change in Employment |
|  NETCHG_EMPLFL_N  | C  | Net Change Noise Flag (G,H,D,S... see INIT_EMPLFL_N)  |
|  BIRTHS_ESTB  | N   |  Number of Establishment Births  |
|  BIRTHS_EMPL  |  N  |  Establishment Births Employment with Noise  |
|   BIRTHS_EMPLFL_N|  C   |  Establishment Births Noise Flag (G,H,D,S... see INIT_EMPLFL_N  |
|  DEATHS_ESTB  |  N   |  Number of Establishment Deaths  |
|  DEATHS_EMPL   |   N  |  Establishment Deaths Employment with Noise  |
|  DEATHS_EMPLFL_N  |  C   |  Establishment Deaths Noise Flag (G,H,D,S... see INIT_EMPLFL_N)  |
|   EXP_ESTB |  N   |  Number of Establishment Expansions  |
|   EXP_EMPL |  N   |   Establishment Expansions Change in Employment with Noise |
|  EXP_EMPLFL_N  |  C   |  Establishment Expansion Noise Flag  |
|  CONTR_ESTB  |   N  |  Number of Establishment Contractions  |
|  CONTR_EMPL  |   N |  Establishment Contractions Change in Employment with Noise  |
|  CONTR_EMPLFL_N  |   C  | Establish Contractors Noise Flag   |
|   PCTCHG_ESTB |  N   |  Percent Change Establishments  |
|  PCTCHG_EMPL  |  N  |   Percent Change in Employment |
|  PCTCHG_BIRTHS_EMPL  |  N   |  Percent Change in Employment Due to Births  |
|  PCTCHG_DEATHS_EMPL  |  N   |   Percent Change in Employment Due to Deaths |
|  PCTCHG_BIRTHS_EXP_EMPL  |  N   |   Percent Change in Employment Due to Births & Expansion |
|  PCTCGH_DEATHS_CONTR_EMPL  | N   | Percent change in Employment Due to Deaths & Contractions    |
|  STATEDSCR  |  C   |  State Description  |
|  NAICSDSCR  |   C  |  NAICS Industry Description  |
|  ENTRSIZEDSCR  |  C   |  Enterprise Employment Size Description  |

This denotes employment size class for data withheld to avoid disclosure (confidentiality) or withheld because data do not meet publication standards.
 
1. A:        0-19
2. B:        20-99
3. C:        100-249
4. E:        250-499
5. F:        500-999
6. G:        1,000-2,499
7. H:        2,500-4,999
8. I:        5,000-9,999
9. J:        10,000-24,999
10. K:        25,000-49,999
11. L:        50,000-99,999
12. M:        100,000 or More

_____________________
Other marks
1. G:        Low noise applied to cell value (0 to < 2%)
2. H:        Medium noise applied to cell value (2 to < 5%)
3. D:        Data withheld and value set to 0 to avoid disclosing data for individual businesses; data are included in higher level totals. 
4. S:       Data withheld and value set to 0 to avoise releasing information that does not meet publication standards; data are included in higher level totals.

Will be treating as targets:
1. Number of Initial Year Establishments
2. Change in Establishments
3. Number of Establishment Births
4. Number of Establishment Deaths
5. Number of Establishment Expansions
6. Number of Establishment Contractions
7. Percent Change Establishments

In [74]:
# split the US data with States and using '7' to filter for industries with less than 500 employees
industry_us_data = industry_data.loc[(industry_data['STATE'] == 0) & (industry_data['ENTRSIZE'] == 7)][['NAICS',
                                                    'INIT_ESTB', 'NETCHG_ESTB', 'BIRTHS_ESTB', 'DEATHS_ESTB', 
                                                'EXP_ESTB', 'CONTR_ESTB', 'PCTCHG_ESTB','NCSDSCR','year' ]]
industry_state_data = industry_data.loc[(industry_data['STATE'] != 0) & (industry_data['ENTRSIZE'] == 7)][['STATE',
                                                 'NAICS', 'INIT_ESTB', 'NETCHG_ESTB', 'BIRTHS_ESTB', 'DEATHS_ESTB', 
                                             'EXP_ESTB', 'CONTR_ESTB', 'PCTCHG_ESTB','NCSDSCR','year', 'STATEDSCR']]

In [75]:
industry_us_data.head(3)

,NAICS,INIT_ESTB,NETCHG_ESTB,BIRTHS_ESTB,DEATHS_ESTB,EXP_ESTB,CONTR_ESTB,PCTCHG_ESTB,NCSDSCR,year
6,--,5572590,57956,587233,529277,1487582,1271341,1.0,Total,2015
14,11,17603,433,2186,1753,4259,3853,2.5,"Agriculture, Forestry, Fishing and Hunting",2015
22,113,7455,145,811,666,2114,1890,1.9,Forestry and Logging,2015


In [94]:
industry_state_data.head(3)

,STATE,NAICS,INIT_ESTB,NETCHG_ESTB,BIRTHS_ESTB,DEATHS_ESTB,EXP_ESTB,CONTR_ESTB,PCTCHG_ESTB,NCSDSCR,year,STATEDSCR
3156,1,--,71519,-5,6202,6207,18952,17554,0.0,Total,2015,Alabama
3164,1,11,717,6,63,57,178,192,0.8,"Agriculture, Forestry, Fishing and Hunting",2015,Alabama
3172,1,21,140,-8,4,12,44,42,-5.7,"Mining, Quarrying, and Oil and Gas Extraction",2015,Alabama


_______________

## Proposal for regression analysis

1. Run a regression on the state data linked to the top industry
2. Run a regression on just the industries for the entire US

## Now looking at states full dataset

In [218]:
# create a 1:1 match between a state and a top industry to add into the bigger state regression
state_industry = pd.read_csv('State to Industry Mapping.csv')
temp_data = pd.read_csv('final_data(industry not added).csv')
temp_data = temp_data.replace('(NA)', None).dropna()
final_data = temp_data.merge(state_industry, on='state', how='left')

In [224]:
feature_data = temp_data.drop(['GeoName', 'year', 'state'], axis=1)
feature_data = feature_data.apply(pd.to_numeric)
feature_data['GDP PC'] = feature_data['Real GDP by state']/feature_data['Population (persons) ']

In [226]:
# normalize the dataset
min_max_scaler = preprocessing.MinMaxScaler()
data_normalized = pd.DataFrame(min_max_scaler.fit_transform(feature_data))
data_normalized.columns = feature_data.columns

In [227]:
#data_normalized['State'] = temp_data['state']
#data_normalized['Year'] = temp_data['year']
contract_establish = data_normalized['contract_establish']
end_establish = data_normalized['end_establish']
expand_establish = data_normalized['expand_establish']
net_change = data_normalized['net_change']
open_establish = data_normalized['open_establish']
total_contract = data_normalized['total_contract']
total_expand = data_normalized['total_expand']
data_normalized = data_normalized.drop(['contract_establish', 'end_establish', 'expand_establish', 'net_change',
                                        'open_establish', 'total_contract', 'total_expand'], axis=1)

In [249]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectFromModel
X = data_normalized
y = open_establish

In [354]:
# Create and fit selector
selector = SelectKBest(f_regression, k=21)
selector.fit(X, y)
# Get idxs of columns to keep
idxs_selected = selector.get_support(indices=True)
kbest_scores = pd.DataFrame(selector.scores_)
kbest_scores.columns = ['kbest_score']

In [353]:
m = X.corr()
corr = m.stack().reset_index()
corr.columns = ['var1','var2','corr']
corr_drop = corr.loc[(corr['corr']>0.94) & (corr['corr']<1.00)]
columns = pd.DataFrame(X.columns)
columns.columns = ['var1']
kbest_scores = columns.join(kbest_scores)
corr_drop = corr_drop.merge(kbest_scores, on='var1', how='left')[['var1', 'var2','corr','kbest_score']]
kbest_scores['var2'] = kbest_scores['var1']
corr_drop = corr_drop.merge(kbest_scores, on='var2', how='left')[['var1_x', 'var2',
                                                                  'corr','kbest_score_x', 'kbest_score_y']]

In [360]:
corr_drop['var1_drop'] = corr_drop['kbest_score_x']<corr_drop['kbest_score_y']
list1 = corr_drop.loc[corr_drop['var1_drop']==True]['var1_x']
list2 = corr_drop.loc[corr_drop['var1_drop']==False]['var2']

In [367]:
list1 = list1.drop_duplicates()
list2 = list2.drop_duplicates()
final_drop = pd.concat([list1,list2]).reset_index().drop(['index'], axis=1)

In [368]:
final_drop

,0
0,Gross domestic product (GDP) by state
1,Gross operating surplus
2,Per capita real GDP by state
3,Real GDP by state
4,pce Financial services and insurance
5,pce Household consumption expenditures (for s...
6,pce Goods
7,pce Less: Receipts from sales of goods and ser...
8,pce Services
9,Per capita personal income (dollars)


In [ ]:
# Create new dataframe with only desired columns, or overwrite existing
X_new = X.iloc[:,idxs_selected]

In [239]:
from sklearn.feature_selection import SelectFromModel
lsvc = linear_model.LinearRegression().fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
idxs_selected = model.get_support(indices=True)
# Create new dataframe with only desired columns, or overwrite existing
X_new = X.iloc[:,idxs_selected]

In [214]:
X_test = X_new.drop(['Real GDP by state', 'Population (persons) ', 
                     'Employer contributions for government social insurance', 'Total employment',
                    'Employee and self-employed contributions for government social insurance'],axis=1)

In [246]:
X_test = X_new.drop(["Equals: Farm proprietors' income"], axis=1)

In [247]:
lm = linear_model.LinearRegression()
lm.fit(X_test, y)
lm.score(X_test,y)

0.7693142849910656

In [198]:
X_new = X_new.drop(['Imputed interest receipts ', 'Nonfarm proprietors employment', 'Proprietors employment',
                    'Wage and salary employment', 'Total employment (number of jobs)', 
                    'Income taxes (net of refunds) '], axis=1)

In [248]:
model = sm.OLS(y, X_new).fit()
predictions = model.predict(X_new) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         open_establish   R-squared:                       0.849
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     896.6
Date:                Sun, 18 Nov 2018   Prob (F-statistic):               0.00
Time:                        00:10:11   Log-Likelihood:                 1341.5
No. Observations:                 966   AIC:                            -2671.
Df Residuals:                     960   BIC:                            -2642.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Farm proprietors' income                         1.5735      0.384      4.093      0.000       0.819       2.328
Nonfarm proprietors' income                      0.4935      0.022     22.731      0.000       0.451       0.536
Less: Net income of corporate farms              2.1004      0.564      3.723      0.000       0.993       3.208
Plus: Farm wages and salaries                   -1.0773      0.625     -1.724      0.085      -2.303       0.149
Equals: Farm earnings                            4.2640      1.810      2.356      0.019       0.712       7.816
Equals: Farm proprietors' income                 1.5735      0.384      4.093      0.000       0.819       2.328
Equals: Net income including corporate farms    -7.6382      2.369     -3.224      0.001     -12.287      -2.989
==============================================================================
Omnibus:                      416.651   Durbin-Watson:                   1.588
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3727.875
Skew:                           1.738   Prob(JB):                         0.00
Kurtosis:                      11.975   Cond. No.                     1.81e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.73e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## Running General Regression

In [ ]:
# normalizing the dataset
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X_normalized = pd.DataFrame(np_scaled)
X_normalized.head()

In [ ]:
#using sklearn instead of stats models
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)

In [ ]:
predictions = lm.predict(X)
#print(predictions)[0:5]

In [ ]:
lm.score(X,Y)

In [ ]:
lm.coef_

### What if we use open stablish?

In [ ]:
Y = data['open_establish']
X = data.drop(['open_establish', 'state', 'GeoName', 'year', 'Compensation of employees',
              'Gross operating surplus', 'Subsidies', 'Taxes on production and imports', 
               'Taxes on production and imports less subsidies', 'Value of inventory change', 
               'Unemployment compensation for railroad employees', 'Other assistance to veterans 1',
               'Corn', 'Oats', 'Other grains', 'Soybeans', 'Forest and maple products', 'Fruits and nuts',
               'Total grains', 'Plus: Statistical adjustment', 'Plus: Value of inventory change',
               'Value of inventory change: crops', 'Value of inventory change: livestock', 
               'Value of inventory change: materials and supplies'
              ], axis=1)

In [ ]:
# inout issues with compensations of employees, gross operating surplus, subsidies, taxes on production/less subsidies
X = X.iloc[:, 0:20]

In [ ]:
# normalizing the dataset
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X_normalized = pd.DataFrame(np_scaled)
X_normalized.head()

In [ ]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()